In [1]:
# 19-05-2021  # unknown error tabh aata hai pdf mai url sahi honey ke baad v jabh uska naam sahi sey save nhi hota  url name mai

In [ ]:
 # pdf ke download mai jabh unknown error ata hai so most probably wah naam ke wajah sey wajah sey ata hai ki file ka naam sahi dhang sey save nhi ho rha hai , jaise usmai .pdf naam sey nhi save ho rhahai
    # esmai script mai wahi huwa ,unknown error aaaaaaa rha hai while downloading pdf 

In [7]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *

# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'criepi_denken_or_jp_publication'
site_name = 'Central Research Institute of Electric Power Industry (CRIEPI) (JAPAN)'
c = Crawl()  # creating obje_
# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# creating pdf directories
pdf_directory = c.create_pdf_directories(project_path, site)
# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ....\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_download_pdf = 0
publish_source = 'criepi.denken.or.jp'
country = 'Japan'
language = 'English'
images_path = []

home_page = c.download_page('https://criepi.denken.or.jp/en/')

for i in home_page.split('<dl>')[1:]:
    # source_link
    source_link = c.scrap('<a\s*href="(.*?)"', i)  
    if 'http' in source_link:
        continue
    if 'http' not in source_link:
        source_link = 'https://criepi.denken.or.jp' + source_link
    if 'en/enic/index.html' in source_link:
        continue
    print(source_link + "_" , count)
    continue
    # handle duplicates
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    if dic:
        duplicate_data += 1
        continue

    time.sleep(random.randint(1,3))

    logger.info(f'Fetching {source_link}\n')
    if '.pdf' not in source_link:
        page = c.download_page(source_link)   
        if page.startswith('Unable to fetch'):
            logger.info(page)
            unable_to_fetch_article_url += 1
            continue 
    if '.pdf' in source_link:
#         source_headline = c.scrap('/(.*?).pdf',source_link)  # in this it wil take the first match (/)
        source_headline = c.scrap('.*/(.*?).pdf',source_link)  # it wil match the last match of the condition
    else:        
        source_headline = c.scrap('<title>(.*?)</title>', page)
        source_headline = re.sub('&amp;','&',source_headline)
        source_headline = re.sub('^CRIEPI|-|About CRIEPI','',source_headline).strip()  # to subsitute the starting of the occurance


    # skip if headline not found
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue

     # Date and time
    pub_date, publish_time = created_on , current_time

    # skip null date
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')            
        skipped_due_to_date += 1
        continue

    # break if date is not today's date
    if pub_date != created_on:
        break    

    # source_content
    if '.pdf' not in source_link:
        source_content = c.scrap('<div\s*class="privacy-Box">(.*?)<p\s*class="page-top">',page)
        if not source_content:
            source_content = source_headline          
    else:
        source_content = source_headline
    source_content = re.sub('&.*?;','',source_content,re.S)
    source_content = c.strip_html(source_content)

    if not source_content:
        logger.info(f'Skipping due to content {source_link}\n')            
        skipped_due_to_content += 1
        continue
    journalist = ''
    if '.pdf' not in source_link:
        journalist =c.scrap("'author-name':'(.*?)'",page)

    if not journalist: journalist = 'NA'

    # current date and time 00
    harvest_time = datetime.now().strftime("%H:%M:%S")

    # temp link
    temp_link = source_link

    # headline and content 
    headline = source_headline
    content = source_content

    # overall_tonality
    overall_tonality = ''

    # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content = ''

    # image_urls
    image_urls = []
    
    pdf_path, pdf_name = '', ''

    if '.pdf' in source_link:
        pdf_url = source_link
        pdf_name = c.scrap('.*\/(.*)\?',pdf_url)  # agar name dhang sey nhi save hoga mean .pdf ending mai so unknown error ata hai mostly
        pdf_path = f'{pdf_directory}/{pdf_name}'        
        # download pdf
        pdf = c.download_pdf(pdf_url, pdf_path)
        if pdf.startswith('Unable to fetch'):
            logger.info(pdf) # writes error message with error code
            unable_to_download_pdf += 1
            continue
    else:
        for i in page.split('href=')[1:]:
            pdf_url = c.scrap('"(.*?)"',i)
            if '.pdf' in pdf_url:
                if ('komae.pdf' in pdf_url) or ('nuclear.pdf' in pdf_url) or ('system.pdf' in pdf_url):
                    continue
                if 'http' not in pdf_url:
                    pdf_main_url = c.scrap('(.*)/',source_link)
                    pdf_url = pdf_main_url + '/'+ pdf_url
                    if '?' in pdf_url:
                        pdf_name = c.scrap('.*\/(.*)\?',pdf_url)
                    else:
                        pdf_name = c.scrap('.*\/(.*)',pdf_url)
                # pdf_path
                pdf_path = f'{pdf_directory}/{pdf_name}'        

                # download pdf
                pdf = c.download_pdf(pdf_url, pdf_path)
                if pdf.startswith('Unable to fetch'):
                    logger.info(pdf) # writes error message with error code
                    unable_to_download_pdf += 1
                    continue
  
    # storing the above data in a dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":'',
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content,
                    "pdf_url": pdf_url,
                    "pdf_name": pdf_name,
                    "pdf_path":pdf_path
                }

#     cl_data.insert_one(clientdata)  
    no_of_data += 1
logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Unable to download pdf: {unable_to_download_pdf}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n') 

/en/aboutcriepi/outline.html_ 0
/en/publications/pamphletslabo/index.html_ 1
/en/aboutcriepi/privacy.html_ 2
/en/enic/index.html_ 3
/en/publications/pamphletslabo/index.html_ 4
/en/publications/pamphletslabo/index.html_ 5
/en/aboutcriepi/outline.html_ 6
/en/aboutcriepi/outline.html_ 7
/en/publications/pamphletslabo/index.html_ 8
/en/aboutcriepi/FY2018_AnnualReport.pdf?v2_ 9


In [10]:
import re
a = 'https://criepi.denken.or.jp/en/aboutcriepi/FY2018_AnnualReport.pdf?v2'
name = re.search('.*/(.*?).pdf',a).group(1)
print(name)

FY2018_AnnualReport
